# Neo4j: Application of graph algorithms
COURSE: INFO9016-1 Advanced Databases

AUTHORS: 
- Emilien DE LA BRASSINE BONARDAUX 
- Yanis GEURTS
- Eri VAN DE VYVER


## Setup

The utilization of this notebook require having docker install. 

Execute these lines : 

`docker pull neo4J`

`docker compose up`

In [160]:
!pip install neo4j
from neo4j import GraphDatabase
import pandas as pd

In [161]:
# --- Define Connection Parameters ---
# The hostname 'adb-neo4j' resolves correctly inside the Jupyter container
URI = "bolt://localhost:7687"

# Authentication is an empty tuple because NEO4J_AUTH: none is set in docker-compose.yml
AUTH = ("neo4j", "neo4j")  # Default credentials for Neo4j when no auth is set

# --- Establish Connection and Query Executor ---
try:
    driver = GraphDatabase.driver(URI, auth=AUTH)
    driver.verify_connectivity()
    print("Connection to Neo4j successful! Ready to load data.")
except Exception as e:
    print(f"Connection failed. Please check if your 'adb-neo4j' container is running. Error: {e}")

def run_cypher_query(query):
    """Executes a Cypher query in a write transaction (suitable for DDL and data loading)."""
    with driver.session() as session:
        # Use execute_write for CREATE/MERGE operations
        result = session.execute_write(lambda tx: tx.run(query).data())
        return result


Connection to Neo4j successful! Ready to load data.


In [162]:
print("\n--- Step 1: Creating Constraints and Indexes ---")

CONSTRAINTS_AND_INDEXES = [
    "CREATE CONSTRAINT airports IF NOT EXISTS FOR (a:Airport) REQUIRE a.iata IS UNIQUE",
    "CREATE CONSTRAINT cities IF NOT EXISTS FOR (c:City) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT regions IF NOT EXISTS FOR (r:Region) REQUIRE r.name IS UNIQUE",
    "CREATE CONSTRAINT countries IF NOT EXISTS FOR (c:Country) REQUIRE c.code IS UNIQUE",
    "CREATE CONSTRAINT continents IF NOT EXISTS FOR (c:Continent) REQUIRE c.code IS UNIQUE",
    "CREATE INDEX locations IF NOT EXISTS FOR (air:Airport) ON (air.location)"
]

try:
    for query in CONSTRAINTS_AND_INDEXES:
        run_cypher_query(query)
        
    print("Constraints and indexes created successfully.")

except Exception as e:
    print(f"Failed to create constraints/indexes: {e}")


print("\n--- Step 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv ---")

NODE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///airport-node-list.csv' AS row

MERGE (a:Airport {iata: row.iata})
MERGE (ci:City {name: row.city})
MERGE (r:Region {name: row.region})
MERGE (co:Country {code: row.country})
MERGE (con:Continent {name: row.continent})

MERGE (a)-[:IN_CITY]->(ci)
MERGE (a)-[:IN_COUNTRY]->(co)
MERGE (ci)-[:IN_COUNTRY]->(co)
MERGE (r)-[:IN_COUNTRY]->(co)
MERGE (a)-[:IN_REGION]->(r)
MERGE (ci)-[:IN_REGION]->(r)
MERGE (a)-[:ON_CONTINENT]->(con)
MERGE (ci)-[:ON_CONTINENT]->(con)
MERGE (co)-[:ON_CONTINENT]->(con)
MERGE (r)-[:ON_CONTINENT]->(con)

SET a.id = row.id,
    a.icao = row.icao,
    a.descr = row.descr,
    a.runways = toInteger(row.runways),
    a.longest = toInteger(row.longest),
    a.altitude = toInteger(row.altitude),
    a.latitude = toFloat(row.lat),
    a.longitude = toFloat(row.lon)
    // Removed any potential a.city property to keep city as a separate City label

RETURN count(a) AS airports_loaded
"""

results_nodes = run_cypher_query(NODE_LOADING_QUERY)
print(f"Airports and Geographies loaded. Total Airports: {results_nodes[0]['airports_loaded']}")

print("\n--- Step 3: Loading Routes from iroutes-edges.csv ---")

ROUTE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///iroutes-edges.csv' AS row

MATCH (source:Airport {iata: row.src})
MATCH (target:Airport {iata: row.dest})

MERGE (source)-[r:HAS_ROUTE]->(target)
ON CREATE SET r.distance = toFloat(row.dist)

RETURN count(r) AS routes_loaded
"""

results_routes = run_cypher_query(ROUTE_LOADING_QUERY)
print(f"Routes loaded. Total HAS_ROUTE relationships: {results_routes[0]['routes_loaded']}")

def run_query(query, params={}):
    """
    Executes a Cypher query using the global 'driver'
    and returns the result as a pandas DataFrame.
    """
    try:
        df = driver.execute_query(
            query,
            parameters_=params,
            database_="neo4j", 
            result_transformer_=pd.DataFrame
        )
        return df
    
    except Exception as e:
        print(f"Query failed: {e}")
        return pd.DataFrame()


--- Step 1: Creating Constraints and Indexes ---
Constraints and indexes created successfully.

--- Step 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv ---
Airports and Geographies loaded. Total Airports: 3503

--- Step 3: Loading Routes from iroutes-edges.csv ---
Routes loaded. Total HAS_ROUTE relationships: 46389


## Introduction

### What is Neo4j ? 

### What are graph algorithms

## Overview

### Features

### Benefits and Drawback vs Relational Databases

### Real-World Use Cases

## Running Graph Algorithms


### Presentation of the dataset

The dataset that we are going to use regroups ...

### Simple query : Explorating Data

In [163]:
# --- Data Exploration ---
# Explore basic statistics
with driver.session() as session:
    num_nodes = session.run("MATCH (n) RETURN count(n) AS c").single()["c"]
    num_rels = session.run("MATCH ()-[r]->() RETURN count(r) AS c").single()["c"]

print(f"Nombre de nœuds : {num_nodes}")
print(f"Nombre de relations : {num_rels}")

with driver.session() as session:
    labels = session.run("CALL db.labels()")
    print("Labels disponibles :")
    for record in labels:
        print("-", record["label"])


Nombre de nœuds : 8627
Nombre de relations : 73954
Labels disponibles :
- Airport
- City
- Region
- Country
- Continent


In [164]:
# Explorate some relationships
with driver.session() as session:
    rels = session.run("CALL db.relationshipTypes()")
    print("Types de relations :")
    for record in rels:
        print("-", record["relationshipType"])

Types de relations :
- IN_CITY
- IN_COUNTRY
- IN_REGION
- ON_CONTINENT
- HAS_ROUTE


In [165]:
# Explorate some nodes
label = "Airport" 
with driver.session() as session:
    result = session.run(f"MATCH (n:{label}) RETURN n LIMIT 10")
    df = pd.DataFrame([dict(record["n"]) for record in result])  
df

,altitude,descr,longest,iata,latitude,icao,id,runways,longitude
0,1026,Hartsfield - Jackson Atlanta International Air...,12390,ATL,33.636700,KATL,1,5,-84.428101
1,151,Anchorage Ted Stevens,12400,ANC,61.174400,PANC,2,3,-149.996002
2,542,Austin Bergstrom International Airport,12250,AUS,30.194500,KAUS,3,2,-97.669899
3,599,Nashville International Airport,11030,BNA,36.124500,KBNA,4,4,-86.678200
4,19,Boston Logan,10083,BOS,42.364300,KBOS,5,6,-71.005203
5,143,Baltimore/Washington International Airport,10502,BWI,39.175400,KBWI,6,3,-76.668297
6,14,Ronald Reagan Washington National Airport,7169,DCA,38.852100,KDCA,7,3,-77.037697
7,607,Dallas/Fort Worth International Airport,13401,DFW,32.896801,KDFW,8,7,-97.038002
8,64,Fort Lauderdale/Hollywood International Airport,9000,FLL,26.072599,KFLL,9,2,-80.152702
9,313,Washington Dulles International Airport,11500,IAD,38.944500,KIAD,10,4,-77.455803


In [166]:
# Explorate some relationships
with driver.session() as session:
    result = session.run("""
        MATCH (a:Airport)-[r:IN_CITY]->(b:City)
        RETURN a.iata AS source, b.name AS target, type(r) AS relation
        LIMIT 10
    """)
    df_rels = pd.DataFrame([record.data() for record in result])

df_rels 

,source,target,relation
0,ATL,Atlanta,IN_CITY
1,ANC,Anchorage,IN_CITY
2,AUS,Austin,IN_CITY
3,BNA,Nashville,IN_CITY
4,BOS,Boston,IN_CITY
5,BWI,Baltimore,IN_CITY
6,IAD,Washington D.C.,IN_CITY
7,DCA,Washington D.C.,IN_CITY
8,DAL,Dallas,IN_CITY
9,DFW,Dallas,IN_CITY


### Using the GDS library

Typically, when one want to use graph algorithms, the Graph Data Science library is the best option. Indeed, it has multiple advantages : 

- 


In cypher, the way to call a graph algorithm with GDA is as follow : 

`CALL gds[.<tier>].<algorithm>.<execution-mode>[.<estimate>](`

`graphName: String,`

  `configuration: Map)`

### The PageRank Algorithm

For this first algorithm, we are going to apply the PageRank algorithm on a graph containing the different country with their airport. This will allow us to get the most important country, which is determined by considering links with other country that are considered more important.

To do that, we start by creating a projected graph with the different country as note and the relationship between two country as relationship. This last relation is derived from the relation linking two country. These kind of request are one the main advantages of Cypher.

The second step is to apply the PageRank algorithm on this graph. Once it is done, we are taking the nodeId and score of each node. With this, we can order the country by score descending (resp. ascending) with the keyword DESC (resp. ASC).

In [ ]:
# Let's now create a projected graph with only the needed nodes and relationships
with driver.session() as session:
    #run_query("CALL gds.graph.drop('country_routes') YIELD graphName")
    result = run_query("""
        CALL gds.graph.project.cypher(
        'country_routes', 
        'MATCH (c:Country) RETURN id(c) AS id',
        'MATCH (c1:Country)<-[:IN_COUNTRY]-(a1:Airport)-[:HAS_ROUTE]->(a2:Airport)-[:IN_COUNTRY]->(c2:Country) RETURN id(c1) AS source, id(c2) AS target'
        )
        YIELD graphName, nodeCount, relationshipCount
    """)
    print(result)
    

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. gds.graph.project.cypher is deprecated. It is replaced by gds.graph.project Cypher projection as an aggregation function.', position=<SummaryInputPosition line=2, column=9, offset=9>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 9, 'line': 2, 'column': 9}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n        CALL gds.graph.project.cypher(\n        'country_routes', \n        'MATCH (c:Country) RETURN id(c) AS id',\n        'MATCH (c1:Country)<-[:IN_COUNTRY]-(a1:Airport)-[:HAS_ROUTE]->(a2:Airport)-[:IN_COUNTRY]->(c2:Country) RETURN id(c1) AS source, id(c2) AS target'\n        )\n    

                0    1      2
0  country_routes  232  46389


In [154]:
with driver.session() as session:
    result = run_query("""
    CALL gds.pageRank.stream('country_routes')
    YIELD nodeId, score

    WITH gds.util.asNode(nodeId) AS country, score
    RETURN country.code AS Country, score
    ORDER BY score ASC
    LIMIT 10
    """)
    print(result)

    0         1
0  AS  0.152342
1  CX  0.154961
2  FK  0.155041
3  SH  0.160565
4  SZ  0.160565
5  LS  0.160565
6  PM  0.161198
7  KP  0.161556
8  FO  0.162528
9  TL  0.163046


### PathFinding Algortihm: A*
The A* algorithm is an algorithm that is made to compute the shortest path between two nodes in a graph with weighted relationships. We will try to use it to find the shortest path between two airports.

Let's first create a projected graph that contains the airports and their connections. The A* algorithm will need to access the latitude and longitude of the airport to compute the heuristic. 



In [ ]:
# Let's now create a projected graph with only the needed nodes and relationships
with driver.session() as session:
    #run_query("CALL gds.graph.drop('airport_relations') YIELD graphName")
    result = run_query("""
    CALL gds.graph.project.cypher(
        'airport_relations',
        'MATCH (a:Airport)
        RETURN id(a) AS id,
                a.latitude AS latitude, 
                a.longitude AS longitude',
        'MATCH (a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)
        RETURN id(a1) AS source, id(a2) AS target, 
                r.distance AS distance'
    )
    YIELD graphName, nodeCount, relationshipCount
    """)
    print(result)

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. gds.graph.project.cypher is deprecated. It is replaced by gds.graph.project Cypher projection as an aggregation function.', position=<SummaryInputPosition line=2, column=5, offset=5>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 5, 'line': 2, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    CALL gds.graph.project.cypher(\n        'airport_relations',\n        'MATCH (a:Airport)\n        RETURN id(a) AS id,\n                a.latitude AS latitude, \n                a.longitude AS longitude',\n        'MATCH (a1:Airport)-[r:HAS_ROUTE]->(a2:Airport)\n        RETURN id(a1)

                   0     1      2
0  airport_relations  3503  46389


The next step is to use the Algorithm with our graph. For the purpose of the example, we will try to use two city that are poorly connected. We are able to find them by using the PageRank algorithm that is shown above. We are going to try to find the shortest path between Pago Pago and Mount Pleasant. Both are located in small island, the first one in Oceania and the second near Argentina. 

In [169]:
with driver.session() as session:
    result = run_query("""
    MATCH (source:Airport {iata : 'PPG'}), (target:Airport {iata: 'MPN'})
    CALL gds.shortestPath.astar.stream('airport_relations', {
        sourceNode: id(source),
        targetNode: id(target),
        latitudeProperty: 'latitude',
        longitudeProperty: 'longitude',
        relationshipWeightProperty: 'distance'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).iata AS sourceNodeName,
        gds.util.asNode(targetNode).iata AS targetNodeName,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeNames,
        costs
    ORDER BY index
    """)
    print(result)

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=4, column=21, offset=157>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 157, 'line': 4, 'column': 21}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n    MATCH (source:Airport {iata : 'PPG'}), (target:Airport {iata: 'MPN'})\n    CALL gds.shortestPath.astar.stream('airport_relations', {\n        sourceNode: id(source),\n        targetNode: id(target),\n        latitudeProperty: 'latitude',\n        longitudeProperty: 'longitude',\n        relationshipWeight

     0    1        2                                    3  \
0  PPG  MPN  12202.0  [PPG, HNL, PPT, IPC, SCL, PUQ, MPN]   

                                                   4  
0  [0.0, 2610.0, 5352.0, 7990.0, 10320.0, 11674.0...  


The output shows us that the algorithm worked well and show us the shortest path between both airports. 

### SUITE ? 

In [157]:
driver.close()
print("\nNeo4j connection closed.")


Neo4j connection closed.
